In [ ]:
# Install required dependencies, as needed

### %pip install mysqlx
### %pip install kmodes

In [ ]:
# Import packages for data connection and pre-processing 

import mysqlx
import numpy as np
import mysql.connector
import pandas as pd
import warnings as wrn
from textblob import TextBlob
import nltk
nltk.download('punkt')

# Import visualization package 

import matplotlib.pyplot as plt

# Import packages for clustering 

from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes
from sklearn.cluster import KMeans

# Hide warning messages 

wrn.filterwarnings('ignore')

In [ ]:
# Import data from MySQL database

dbcon = mysql.connector.connect(
host="68.66.233.182",
port=3306,
user="vordonez",
password="25300476")

# Create dataframes per SQL query

try:
    SQL_leads = pd.read_sql_query(
        '''SELECT * FROM `H_Olist`.`leads`''', dbcon)
    
    SQL_merchants = pd.read_sql_query(
          '''SELECT * FROM `H_Olist`.`merchants`''', dbcon)
    
    SQL_merchant_saas_won = pd.read_sql_query(
          '''SELECT * FROM `H_Olist`.`merchant_saas_won`''', dbcon)
    
    SQL_order_line_items = pd.read_sql_query(
          '''SELECT * FROM `H_Olist`.`order_line_items`''', dbcon)
    
    SQL_reviews = pd.read_sql_query(
          '''SELECT * FROM `H_Olist`.`reviews`''', dbcon)
    
    SQL_reviews_bis = pd.read_sql_query(
          '''SELECT * FROM `H_Olist`.`reviews_bis`''', dbcon)
    
    leads = pd.DataFrame(SQL_leads,columns=['lead_id','registered_on_landing_page_date',
                                           'landing_page_id','lead_type','origin'])

    merchants = pd.DataFrame(SQL_merchants, columns=['merchant_id','lead_id','sdr_id','sr_id',
                                          'business_segment','lead_behavior',
                                          'has_company','has_gtn','average_stock',
                                          'business_type','declared_product_catalog_size',
                                          'declared_monthly_revenue'])
    
    merchant_saas_won = pd.DataFrame(SQL_merchant_saas_won, columns=['merchant_id',
                                                                     'won_date_ie_date_of_first_saas_payment'])
    
    order_line_items = pd.DataFrame(SQL_order_line_items,columns=['order_id','order_line_item','product_id',
                                                                 'seller_id','order_date_time','price','freight_cost'])

    reviews = pd.DataFrame(SQL_reviews,columns=['review_id','order_id',
                                           'review_score','review_creation_date','review_answer_timestamp'])
    
    reviews_bis = pd.DataFrame(SQL_reviews_bis,columns=['review_id','order_id',
                                           'review_comment_title','review_creation_date','review_answer_timestamp'])
    
except:
    print("Error: unable to convert the data")

# Close database connection
dbcon.close()

In [ ]:
# Examine contents of reviews

reviews.head()

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp
0,"﻿""7bc2406110b926393aa56f80a40eba40""",73fc7af87114b39712e6da79b0a377eb,4,2021-01-18 00:00:00,2021-01-18 21:46:59
1,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,2021-01-18 00:00:00,2021-01-18 21:46:59
2,97ca439bc427b48bc1cd7177abe71365,00010242fe8c5a6d1ba2dd792cb16214,5,2020-09-21 00:00:00,2020-09-22 10:57:03
3,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,2021-03-10 00:00:00,2021-03-11 03:05:13
4,1fac9acb29e014e3b9b393d5c8505f99,0188777fe321843a18be24a6e9aa1e53,5,2020-07-29 00:00:00,2020-08-01 20:55:50


In [ ]:
# Examine contents of reviews_bis

reviews_bis.head()

,review_id,order_id,review_comment_title,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,,NaN,NaN
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,,NaN,NaN
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,,NaN,NaN
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,,NaN,NaN
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,,NaN,NaN


In [ ]:
# Clean IDs for reviews and reviews_bis

reviews['review_id'] = reviews['review_id'].str.replace('"',"")
reviews['order_id'] = reviews['order_id'].str.replace('"',"")
reviews_bis['review_id'] = reviews_bis['review_id'].str.replace('"',"")
reviews_bis['order_id'] = reviews_bis['order_id'].str.replace('"',"")

In [ ]:
# Detect non-encoded characters and re-encode back into ASCII

reviews['review_id'] = reviews['review_id'].str.encode('ascii', 'ignore').str.decode('ascii')
reviews['order_id'] = reviews['order_id'].str.encode('ascii', 'ignore').str.decode('ascii')
reviews_bis['review_id'] = reviews_bis['review_id'].str.encode('ascii', 'ignore').str.decode('ascii')
reviews_bis['order_id'] = reviews_bis['order_id'].str.encode('ascii', 'ignore').str.decode('ascii')

In [ ]:
# Drop duplicate columns

reviews_bis.drop(['review_creation_date','review_answer_timestamp'],axis=1,inplace=True)

In [ ]:
# Create a merged reviews dataframe

reviews_merged = pd.merge(reviews,reviews_bis,on=['review_id','order_id'],how='left')
reviews_merged.drop_duplicates(keep='first',inplace=True)
reviews_merged.drop('review_answer_timestamp',axis=1,inplace=True)

In [ ]:
# Check if merge was successful

reviews_merged.head()

,review_id,order_id,review_score,review_creation_date,review_comment_title
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,2021-01-18 00:00:00,
2,97ca439bc427b48bc1cd7177abe71365,00010242fe8c5a6d1ba2dd792cb16214,5,2020-09-21 00:00:00,
3,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,2021-03-10 00:00:00,
4,1fac9acb29e014e3b9b393d5c8505f99,0188777fe321843a18be24a6e9aa1e53,5,2020-07-29 00:00:00,
5,2eab04a8f45efa131d44cf58b37ab595,0191bbb5166d4b9a68df11797d5acfa9,5,2020-08-25 00:00:00,


In [ ]:
# Detect non-encoded characters and re-encode back into ASCII

leads['lead_id'] = leads['lead_id'].str.encode('ascii', 'ignore').str.decode('ascii')
leads['landing_page_id'] = leads['landing_page_id'].str.encode('ascii', 'ignore').str.decode('ascii')
leads.drop_duplicates(keep='first',inplace=True)

In [ ]:
# Convert costs from non-numeric to numeric values

order_line_items['freight_cost'] = pd.to_numeric(order_line_items['freight_cost'])
order_line_items['price'] = pd.to_numeric(order_line_items['price'])

In [ ]:
# Detect non-encoded characters and re-encode back into ASCII

order_line_items['order_id'] = order_line_items['order_id'].str.encode('ascii', 'ignore').str.decode('ascii')
order_line_items['product_id'] = order_line_items['product_id'].str.encode('ascii', 'ignore').str.decode('ascii')
order_line_items['seller_id'] = order_line_items['seller_id'].str.encode('ascii', 'ignore').str.decode('ascii')

order_line_items['order_id'] = order_line_items['order_id'].str.replace('"',"")
order_line_items['product_id'] = order_line_items['product_id'].str.replace('"',"")
order_line_items['seller_id'] = order_line_items['seller_id'].str.replace('"',"")

In [ ]:
# Create an grouped orders dataframe by seller id

olt_fin = order_line_items.groupby('seller_id').mean().reset_index()

In [ ]:
# Check grouped dataframe details

olt_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   seller_id     3095 non-null   object 
 1   price         3095 non-null   float64
 2   freight_cost  3095 non-null   float64
dtypes: float64(2), object(1)
memory usage: 72.7+ KB


In [ ]:
# Convert Portuguese date strings to unified date-time format

leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("'", '')

leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("janeiro", 'january')
leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("fevereiro", 'february')
leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("março", 'march')
leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("mar�o", 'march')
leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("abril", 'april')
leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("maio", 'may')
leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("junho", 'june')
leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("julho", 'july')
leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("agosto", 'august')
leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("setembro", 'september')
leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("outubro", 'october')
leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("novembro", 'november')
leads['registered_on_landing_page_date'] = leads['registered_on_landing_page_date'].str.replace("dezembro", 'december')

leads['registered_on_landing_page_date'] = pd.to_datetime(leads['registered_on_landing_page_date'], format='%B %d, %y')

In [ ]:
# Drop landing page ID

leads.drop('landing_page_id',axis=1,inplace=True)

In [ ]:
# Convert merchant SAAS payments to unified date-time format

merchant_saas_won['won_date_ie_date_of_first_saas_payment'] = merchant_saas_won['won_date_ie_date_of_first_saas_payment'].str.replace(":", '-')
merchant_saas_won['won_date_ie_date_of_first_saas_payment'] = pd.to_datetime(merchant_saas_won['won_date_ie_date_of_first_saas_payment'],format='%Y-%m-%d')

In [ ]:
# Create merged merchants dataframe

merchants_merged = pd.merge(merchants,merchant_saas_won,on='merchant_id',how='left')

In [ ]:
# View created dataframe

merchants_merged.head()

,merchant_id,lead_id,sdr_id,sr_id,business_segment,lead_behavior,has_company,has_gtn,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue,won_date_ie_date_of_first_saas_payment
0,00065220becb8785e2cf78355eb9bf68,00dc5000951c268cc8655294daf67b1b,9d12ef1a7eca3ec58c545c678af7869c,de63de0d10a6012430098db33c679b0b,handcrafted,Stability / Green,False,NaN,20-50,reseller,,0.0,2021-07-10
1,01266d4c46afa519678d16a8b683d325,ae437864746ae2a9e694a6ed2b5824db,4b339f9567d060bcea4f5136b9f5949e,6565aa9ce3178a5caf6171827af3a9ba,construction_tools_house_garden,Stability / Green,,NaN,,reseller,,0.0,2021-05-18
2,013d100f608a64ec8275c038aedeb064,f7066703eb3987f21f0d102ea8caaeb5,a8387c01a09e99ce014107505b92388c,495d4e95a8cf8bbf8b432b612a2aa328,food_supplement,,,NaN,,reseller,,0.0,2021-04-06
3,01fd077212124329bac32490e8ef80d9,180d9ac0990bb42907fe9cc7aa3eb5a1,09285259593c61296eef10c734121d5b,85fc447d336637ba1df43e793199fbc8,household_utilities,Stability / Green,,NaN,,reseller,,0.0,2021-03-28
4,0255dc67cb6bad298606136951ab1877,4388a23da67b154b780b78dd7ea4636e,4b339f9567d060bcea4f5136b9f5949e,6565aa9ce3178a5caf6171827af3a9ba,computers,Conformity / Blue,,NaN,,reseller,,0.0,2021-03-06


In [ ]:
# Drop unneeded columns

merchants_merged.drop(['sdr_id','sr_id','has_company','has_gtn','average_stock','business_type','declared_product_catalog_size','declared_monthly_revenue'],axis=1,inplace=True)

In [ ]:
# Create merged dataframe of leads and merchants 

lead_merch = pd.merge(leads,merchants_merged,on='lead_id',how='left')

In [ ]:
# Create dataframe of earliest sell date per merchant

earliest = order_line_items[['seller_id','order_date_time']].groupby('seller_id').min().reset_index()

In [ ]:
# Join earliest sell date on leads and merchants

lead_merch_2 = pd.merge(lead_merch,earliest,left_on='merchant_id',right_on='seller_id',how='left')

In [ ]:
# View resulting dataframe

lead_merch.head()

,lead_id,registered_on_landing_page_date,lead_type,origin,merchant_id,business_segment,lead_behavior,won_date_ie_date_of_first_saas_payment
0,0002ac0d783338cfeab0b2bdbd872cda,2020-11-14,,unknown,NaN,NaN,NaN,NaT
1,000dd3543ac84d906eae52e7c779bb2a,2021-04-05,industry,organic_search,500b5e25308adf85bbc0bbc52c3dc05b,home_decor,Stability / Green,2021-04-24
2,001d3439223b7bb23ed89b9c8890d096,2021-04-14,,paid_search,NaN,NaN,NaN,NaT
3,00275937100699204a3d5ae3caf7190d,2021-01-11,,organic_search,NaN,NaN,NaN,NaT
4,003d859bbd84a526b9e06340ce7d185f,2021-04-02,,direct_traffic,NaN,NaN,NaN,NaT


In [ ]:
# Change date to unified date-time

lead_merch_2['order_date_time'] = pd.to_datetime(lead_merch_2['order_date_time'], format='%Y-%m-%d')

In [ ]:
# Create first sale column to check for first successful sale

lead_merch_2['first_sale'] = lead_merch_2['order_date_time'] - lead_merch_2['won_date_ie_date_of_first_saas_payment'] 

In [ ]:
# Check resulting dataframe

lead_merch_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 0 to 7999
Data columns (total 11 columns):
 #   Column                                  Non-Null Count  Dtype          
---  ------                                  --------------  -----          
 0   lead_id                                 8000 non-null   object         
 1   registered_on_landing_page_date         8000 non-null   datetime64[ns] 
 2   lead_type                               8000 non-null   object         
 3   origin                                  8000 non-null   object         
 4   merchant_id                             842 non-null    object         
 5   business_segment                        842 non-null    object         
 6   lead_behavior                           842 non-null    object         
 7   won_date_ie_date_of_first_saas_payment  842 non-null    datetime64[ns] 
 8   seller_id                               380 non-null    object         
 9   order_date_time                         3

In [ ]:
# Merge review dataframes  together

reviews_merged = pd.merge(reviews, reviews_bis, on='review_id', how='left')

In [ ]:
# Change dates as necessary

reviews_merged['review_creation_date'] = pd.to_datetime(reviews_merged['review_creation_date'])
reviews_merged['review_answer_timestamp'] = pd.to_datetime(reviews_merged['review_answer_timestamp'])

# Calculating the difference in days
reviews_merged['days_between_review'] = (reviews_merged['review_answer_timestamp'] - reviews_merged['review_creation_date']).dt.days

In [ ]:
# Convert the column to string type
reviews_merged['review_comment_title'] = reviews_merged['review_comment_title'].astype(str)  

# Apply TextBlob sentiment analysis on the 'review_comment_title' column
reviews_merged['polarity'] = reviews_merged['review_comment_title'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [ ]:
# Take out relevant sellers and change review score to integer

for_merge = order_line_items[['seller_id','order_id']]
reviews_merged['review_score'] = reviews_merged['review_score'].astype(int)  

In [ ]:
# Take out relevant reviews

reviews_merged2 = pd.merge(reviews_merged,for_merge,left_on='order_id_x',right_on='order_id',how='left')

In [ ]:
# Remove duplicates

reviews_merged2.drop_duplicates(inplace=True)

In [ ]:
# Create reviews grouped by seller ID

reviews_grouped = reviews_merged2.groupby('seller_id').mean().reset_index()

In [ ]:
# Create final dataframe

data_fin = pd.merge(lead_merch_,reviews_grouped,left_on='merchant_id',right_on='seller_id',how='left')

In [ ]:
data_fin

,lead_id,registered_on_landing_page_date,lead_type,origin,merchant_id,business_segment,lead_behavior,won_date_ie_date_of_first_saas_payment,activation,seller_id_x,order_date_time,first_sale,seller_id_y,price,freight_cost,seller_id,review_score,days_between_review,polarity
0,0002ac0d783338cfeab0b2bdbd872cda,2020-11-14,,unknown,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000dd3543ac84d906eae52e7c779bb2a,2021-04-05,industry,organic_search,500b5e25308adf85bbc0bbc52c3dc05b,home_decor,Stability / Green,2021-04-24,19 days,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001d3439223b7bb23ed89b9c8890d096,2021-04-14,,paid_search,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00275937100699204a3d5ae3caf7190d,2021-01-11,,organic_search,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,003d859bbd84a526b9e06340ce7d185f,2021-04-02,,direct_traffic,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,ffe640179b554e295c167a2f6be528e0,2020-10-09,industry,unknown,ed8cb7b190ceb6067227478e48cf8dde,home_appliances,Conformity / Blue,2021-07-03,267 days,ed8cb7b190ceb6067227478e48cf8dde,2021-07-26 01:50:13,23 days 01:50:13,ed8cb7b190ceb6067227478e48cf8dde,33.735294,14.277059,ed8cb7b190ceb6067227478e48cf8dde,3.4,1.266667,0.044444
7996,ffec39090fa0c85692730fdbd647e34d,2021-04-17,,email,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7997,ffec991bf0f35f7e7688cf0d39c9f9f4,2020-11-23,,unknown,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7998,fff8db9478d2fd72df65a67ee6b62f67,2021-01-20,online_medium,direct_traffic,bdae679a9b282249bc23b9b69dae9a99,construction_tools_house_garden,,2021-01-24,4 days,bdae679a9b282249bc23b9b69dae9a99,2021-02-20 10:50:25,27 days 10:50:25,bdae679a9b282249bc23b9b69dae9a99,55.262500,15.735833,bdae679a9b282249bc23b9b69dae9a99,3.5,1.333333,0.000000


In [ ]:
# Drop unneeded columns

merged_data = data_fin.drop(['lead_id','seller_id_y','seller_id','days_between_review'],axis=1)

In [ ]:
# Rename columns

merged_data = merged_data.rename(columns={'won_date_ie_date_of_first_saas_payment':'saas_payments',
                                  'activation':'landing_to_saas_days',
                                  'order_date_time':'first_order_date',
                                  'first_sale':'saas_to_first_order_days',
                                  'price':'avg_order_cost',
                                  'freight_cost':'avg_freight_cost',
                                  'price_freight_ratio':'price_to_freight_ratio',
                                  'review_score':'avg_review_score',
                                  'polarity':'avg_review_polarity'})

In [ ]:
# Create final merged dataframe

merged_data_fin = merged_data.drop(['seller_id_x'],axis=1)

In [ ]:
# View final dataframe information

merged_data_fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 0 to 7999
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype          
---  ------                           --------------  -----          
 0   registered_on_landing_page_date  8000 non-null   datetime64[ns] 
 1   lead_type                        8000 non-null   object         
 2   origin                           8000 non-null   object         
 3   merchant_id                      842 non-null    object         
 4   business_segment                 842 non-null    object         
 5   lead_behavior                    842 non-null    object         
 6   saas_payments                    842 non-null    datetime64[ns] 
 7   landing_to_saas_days             842 non-null    timedelta64[ns]
 8   first_order_date                 379 non-null    datetime64[ns] 
 9   saas_to_first_order_days         379 non-null    timedelta64[ns]
 10  avg_order_cost                   380 non-null   

In [ ]:
# Create lead clusters 

lead_clusters = merged_data_fin[['lead_type','origin','business_segment','lead_behavior']]

In [ ]:
# Remove NAs to create merchant clusters

merch_clusters = lead_clusters.dropna()

In [ ]:
# Replace empty values with unknowns

merch_clusters = merch_clusters.replace('', 'unknown')

In [ ]:
# View resulting dataset

merch_clusters

,lead_type,origin,business_segment,lead_behavior
1,industry,organic_search,home_decor,Stability / Green
14,online_big,paid_search,car_accessories,Stability / Green
22,online_medium,paid_search,handcrafted,Stability / Green
26,online_medium,paid_search,audio_video_electronics,Stability / Green
33,online_small,unknown,home_decor,Conformity / Blue
...,...,...,...,...
7984,industry,unknown,bed_bath_table,unknown
7985,industry,direct_traffic,car_accessories,Stability / Green
7994,online_medium,organic_search,sports_leisure,Influence / Yellow
7995,industry,unknown,home_appliances,Conformity / Blue


In [ ]:
# Instantiate kmodes clustering algorithm with ideal number of clusters (elbow graph removed)

kmode = KModes(n_clusters=3, init = "Huang", n_init = 5, verbose=1)
clusters_leads = kmode.fit_predict(merch_clusters)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 43, cost: 1944.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 207, cost: 1909.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 38, cost: 1964.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 170, cost: 1906.0
Run 4, iteration: 2/100, moves: 163, cost: 1906.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 227, cost: 1885.0
Run 5, iteration: 2/100, moves: 21, cost: 1885.0
Best run was number 5


In [ ]:
# Add resulting clusters back into merchants list

merch_clusters.insert(0, "Cluster", clusters_leads, True)

In [ ]:
# Change clusters to start at 1

merch_clusters['Cluster'] = merch_clusters['Cluster'] + 1

In [ ]:
# View resultant dataset

merch_clusters

,Cluster,lead_type,origin,business_segment,lead_behavior
1,1,industry,organic_search,home_decor,Stability / Green
14,1,online_big,paid_search,car_accessories,Stability / Green
22,1,online_medium,paid_search,handcrafted,Stability / Green
26,1,online_medium,paid_search,audio_video_electronics,Stability / Green
33,2,online_small,unknown,home_decor,Conformity / Blue
...,...,...,...,...,...
7984,2,industry,unknown,bed_bath_table,unknown
7985,1,industry,direct_traffic,car_accessories,Stability / Green
7994,1,online_medium,organic_search,sports_leisure,Influence / Yellow
7995,2,industry,unknown,home_appliances,Conformity / Blue


In [ ]:
# Isolate clusters

merch_clusters_fin = merch_clusters[['Cluster']]

In [ ]:
# Take out relevant columns and join with clusters

test = merged_data_fin[['merchant_id','lead_type','origin','business_segment','lead_behavior','landing_to_saas_days','saas_to_first_order_days','avg_order_cost','avg_freight_cost','avg_review_score','avg_review_polarity']]

test = test.join(merch_clusters_fin)

In [ ]:
# Remove null clusters

test = test.dropna(subset = ['Cluster'])

In [ ]:
# Remove resultant dataset

test

,merchant_id,lead_type,origin,business_segment,lead_behavior,landing_to_saas_days,saas_to_first_order_days,avg_order_cost,avg_freight_cost,avg_review_score,avg_review_polarity,Cluster
1,500b5e25308adf85bbc0bbc52c3dc05b,industry,organic_search,home_decor,Stability / Green,19 days,NaT,NaN,NaN,NaN,NaN,1.0
14,134285d1f41da5c13a756ee8142c8a4e,online_big,paid_search,car_accessories,Stability / Green,15 days,72 days 13:51:26,393.175000,20.232500,3.333333,0.000000,1.0
22,00065220becb8785e2cf78355eb9bf68,online_medium,paid_search,handcrafted,Stability / Green,106 days,NaT,NaN,NaN,NaN,NaN,1.0
26,8c6d188ef073e289887bc52bc37f3e61,online_medium,paid_search,audio_video_electronics,Stability / Green,192 days,NaT,NaN,NaN,NaN,NaN,1.0
33,528e4514f9339c1c3dd7535fd1a2d8cb,online_small,unknown,home_decor,Conformity / Blue,1 days,NaT,NaN,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7984,07de60b03d150fb3ea09eab5da5e4c8b,industry,unknown,bed_bath_table,,0 days,14 days 01:29:49,79.900000,18.877778,4.333333,0.037037,2.0
7985,7877194d40c496d36a88ae5957c5e7d1,industry,direct_traffic,car_accessories,Stability / Green,7 days,NaT,NaN,NaN,NaN,NaN,1.0
7994,02f623a8eb246f3c5f7c2f96462654e6,online_medium,organic_search,sports_leisure,Influence / Yellow,2 days,14 days 15:30:17,144.954545,18.855227,4.522727,0.022727,1.0
7995,ed8cb7b190ceb6067227478e48cf8dde,industry,unknown,home_appliances,Conformity / Blue,267 days,23 days 01:50:13,33.735294,14.277059,3.400000,0.044444,2.0


In [ ]:
# Isolate in-funnel sellers

funnel_sellers = test[['merchant_id']]

funnel_sellers.drop_duplicates(keep='first',inplace=True)

In [ ]:
# View in-funnel sellers

funnel_sellers

,merchant_id
1,500b5e25308adf85bbc0bbc52c3dc05b
14,134285d1f41da5c13a756ee8142c8a4e
22,00065220becb8785e2cf78355eb9bf68
26,8c6d188ef073e289887bc52bc37f3e61
33,528e4514f9339c1c3dd7535fd1a2d8cb
...,...
7984,07de60b03d150fb3ea09eab5da5e4c8b
7985,7877194d40c496d36a88ae5957c5e7d1
7994,02f623a8eb246f3c5f7c2f96462654e6
7995,ed8cb7b190ceb6067227478e48cf8dde


In [ ]:
# Rename column

funnel_sellers = funnel_sellers.rename(columns={'merchant_id' : 'funnel_seller_id'})

In [ ]:
# View in-funnel sellers

funnel_sellers

,funnel_seller_id
1,500b5e25308adf85bbc0bbc52c3dc05b
14,134285d1f41da5c13a756ee8142c8a4e
22,00065220becb8785e2cf78355eb9bf68
26,8c6d188ef073e289887bc52bc37f3e61
33,528e4514f9339c1c3dd7535fd1a2d8cb
...,...
7984,07de60b03d150fb3ea09eab5da5e4c8b
7985,7877194d40c496d36a88ae5957c5e7d1
7994,02f623a8eb246f3c5f7c2f96462654e6
7995,ed8cb7b190ceb6067227478e48cf8dde


In [ ]:
# Merge in-funnel sellers with funnel orders

funnel_orders = pd.merge(order_line_items,funnel_sellers,left_on='seller_id',right_on='funnel_seller_id',how='left')

In [ ]:
# Drop null in-funnel sellers

funnel_orders = funnel_orders.dropna(subset = ['funnel_seller_id'])

In [ ]:
# Check if sellers were successfully isolated

funnel_orders['funnel_seller_id'].isna().sum()

107606

In [ ]:
# Isolate in-funnel order IDs

funnel_order_ids = funnel_orders[['order_id']]

In [ ]:
# Drop duplicate IDs

funnel_order_ids.drop_duplicates(keep='first',inplace=True)

In [ ]:
# View funnel order IDs

funnel_order_ids

,order_id
8,0005f50442cb953dcd1d21e1fb923495
44,001ac194d4a326a6fa99b581e9a3d963
70,0029f17cf0e7640c5cb6825af681303f
89,00335f75ea6a4455b524a8dd177067f5
93,00345f338696283410b7977d2e3efc89
...,...
112471,ff96d596c25445650eee60b94fa62244
112475,ff977ea68e67d2606fdb5981ae3d9cc5
112539,ffbcbeeac5c9c9495cd729f896bc1d5f
112544,ffc0249fed109d5d056d7c79b7fa7dd9


In [ ]:
# Isolate relevant review columns

funnel_reviews = reviews_merged[['order_id_x','review_score','polarity']]

In [ ]:
# Merge in-funnel order IDs to isolate in-funnel reviews

funnel_reviews = pd.merge(funnel_reviews,funnel_order_ids,left_on='order_id_x',right_on='order_id',how='left')

In [ ]:
# Drop null funnel reviews 

funnel_reviews = funnel_reviews.dropna(subset = ['order_id'])

In [ ]:
# Isolate relevant review information

funnel_reviews = funnel_reviews[['order_id','review_score','polarity']]

In [ ]:
# Reset indices

funnel_reviews.reset_index(inplace=True)

funnel_orders.reset_index(inplace=True)

In [ ]:
# Add clusters onto main dataframe

test = merged_data_fin[['merchant_id','lead_type','origin','business_segment','lead_behavior','landing_to_saas_days','saas_to_first_order_days','avg_order_cost','avg_freight_cost','avg_review_score','avg_review_polarity']]

test = test.join(merch_clusters_fin)

In [ ]:
# Isolate needed information

funnel_leads_merch = test[['merchant_id','Cluster','lead_type','origin','business_segment','lead_behavior','landing_to_saas_days','saas_to_first_order_days']]

In [ ]:
# Remake dataframe

funnel_leads_merch=data_fin[['merchant_id','registered_on_landing_page_date','won_date_ie_date_of_first_saas_payment','order_date_time','lead_type','origin','lead_behavior']]

In [ ]:
# Connect merchants with clusters to in-funnel leads

funnel_leads_merch = funnel_leads_merch.join(merch_clusters_fin)

In [ ]:
# View resulting dataframe

funnel_leads_merch

,merchant_id,registered_on_landing_page_date,won_date_ie_date_of_first_saas_payment,order_date_time,lead_type,origin,lead_behavior,Cluster
0,NaN,2020-11-14,NaT,NaT,,unknown,NaN,NaN
1,500b5e25308adf85bbc0bbc52c3dc05b,2021-04-05,2021-04-24,NaT,industry,organic_search,Stability / Green,1.0
2,NaN,2021-04-14,NaT,NaT,,paid_search,NaN,NaN
3,NaN,2021-01-11,NaT,NaT,,organic_search,NaN,NaN
4,NaN,2021-04-02,NaT,NaT,,direct_traffic,NaN,NaN
...,...,...,...,...,...,...,...,...
7995,ed8cb7b190ceb6067227478e48cf8dde,2020-10-09,2021-07-03,2021-07-26 01:50:13,industry,unknown,Conformity / Blue,2.0
7996,NaN,2021-04-17,NaT,NaT,,email,NaN,NaN
7997,NaN,2020-11-23,NaT,NaT,,unknown,NaN,NaN
7998,bdae679a9b282249bc23b9b69dae9a99,2021-01-20,2021-01-24,2021-02-20 10:50:25,online_medium,direct_traffic,,3.0


In [ ]:
# Reset indices

funnel_reviews.reset_index(inplace=True)
funnel_orders.reset_index(inplace=True)
funnel_leads_merch.reset_index(inplace=True)

In [ ]:
# Rename funnel orders

funnel_orders = funnel_orders.rename(columns={'first_order_date':'order_date','funnel_seller_id':'merchant_id'})

In [ ]:
# Subset funnel orders

funnel_orders = funnel_orders[['order_id','merchant_id','order_date','price','freight_cost']]

In [ ]:
# View resulting funnel orders

funnel_orders

,order_id,merchant_id,order_date,price,freight_cost
0,0005f50442cb953dcd1d21e1fb923495,ba143b05f0110f0dc71ad71b4466ce92,2021-07-06 14:10:56,53.99,11.40
1,001ac194d4a326a6fa99b581e9a3d963,ffc470761de7d0232558ba5e786e57b7,2021-07-10 02:51:10,54.00,8.54
2,0029f17cf0e7640c5cb6825af681303f,d3dcf0604eabf0224fbd5948b5e02f69,2021-05-08 12:14:52,94.90,18.54
3,00335f75ea6a4455b524a8dd177067f5,70c27847eca8195c983ed7e798c56743,2021-05-02 22:10:29,168.80,8.21
4,00345f338696283410b7977d2e3efc89,516e7738bd8f735ac19a010ee5450d8d,2021-08-23 15:11:40,48.90,18.44
...,...,...,...,...,...
5039,ff96d596c25445650eee60b94fa62244,7238a95bb63ef7e498389fb863b834a1,2021-05-04 13:50:07,329.00,127.55
5040,ff977ea68e67d2606fdb5981ae3d9cc5,c013e57c075a06e5b5c48ee03c525719,2021-05-22 20:31:42,42.99,13.71
5041,ffbcbeeac5c9c9495cd729f896bc1d5f,3d3ccf2b2f8134b10dce9dd446f0e075,2021-06-27 18:21:06,82.80,14.89
5042,ffc0249fed109d5d056d7c79b7fa7dd9,7ff588a03c2aeae4fbd23f9ae64b760d,2021-08-15 14:45:15,24.99,7.44


In [ ]:
# Rename columns

funnel_leads_merch = funnel_leads_merch.rename(columns={"registered_on_landing_page_date":"landing_date","won_date_ie_date_of_first_saas_payment":"saas_date","order_date_time":"first_order_date",
                                   "Cluster":"cluster"})

In [ ]:
# Drop unneeded columns

funnel_reviews = funnel_reviews.drop(['level_0','index'],axis=1)
funnel_orders = funnel_orders.drop(['index'],axis=1)
funnel_leads_merch = funnel_leads_merch.drop(['index'],axis=1)

In [ ]:
# View dataframe

funnel_leads_merch

,merchant_id,landing_date,saas_date,first_order_date,lead_type,origin,lead_behavior,cluster
0,NaN,2020-11-14,NaT,NaT,,unknown,NaN,NaN
1,500b5e25308adf85bbc0bbc52c3dc05b,2021-04-05,2021-04-24,NaT,industry,organic_search,Stability / Green,1.0
2,NaN,2021-04-14,NaT,NaT,,paid_search,NaN,NaN
3,NaN,2021-01-11,NaT,NaT,,organic_search,NaN,NaN
4,NaN,2021-04-02,NaT,NaT,,direct_traffic,NaN,NaN
...,...,...,...,...,...,...,...,...
7995,ed8cb7b190ceb6067227478e48cf8dde,2020-10-09,2021-07-03,2021-07-26 01:50:13,industry,unknown,Conformity / Blue,2.0
7996,NaN,2021-04-17,NaT,NaT,,email,NaN,NaN
7997,NaN,2020-11-23,NaT,NaT,,unknown,NaN,NaN
7998,bdae679a9b282249bc23b9b69dae9a99,2021-01-20,2021-01-24,2021-02-20 10:50:25,online_medium,direct_traffic,,3.0


In [ ]:
#funnel_reviews.to_csv('funnel_reviews.csv', index=False)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#funnel_orders.to_csv('funnel_orders.csv', index=False)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#funnel_leads_merch.to_csv('funnel_leads_merch.csv', index=False)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Isolate business segments

business_segs = test[['merchant_id','business_segment']]

In [ ]:
#business_segs.to_csv('business_segs.csv', index=False)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>